In [1]:
import re
import os
import inspect
import json
from pprint import pprint
from glob import glob
import shutil
from tqdm.auto import tqdm

In [2]:
import grape

In [3]:
BLACKLIST = [
    "OptimizerV2"
]

In [4]:
def explore_python(root, module):
    # Hardcoded are safe TODO: decomment, apparenty they arent :)
    if False and getattr(root, "__all__", None) is not None:
        return {
            getattr(root, x)
            for x in root.__all__
        }
    
    return {
        getattr(root, x) 
        for x in dir(root)
        if not x.startswith("_") 
            and (
                inspect.isfunction(getattr(root, x))
                or inspect.isclass(getattr(root, x)) 
                or inspect.ismodule(getattr(root, x))
            ) and (
                module.__name__ == "grape"
                or
                module.__name__ in inspect.getmodule(getattr(root, x)).__name__
            ) and 
            not inspect.isbuiltin(getattr(root, x))
    }

def recursive_explore_python(root):
    if inspect.isfunction(root) or inspect.isclass(root):
        return {root}
    
    return {
        y
        for x in explore_python(root, root)
        for y in recursive_explore_python(x)
    }

In [5]:
def explore_on_level(module, root):
    return {
        os.path.join(*root, x) + ".py":getattr(module, x)
        for x in dir(module)
        if not x.startswith("_")
    }

In [6]:
def tabbify(string):
    return "\n".join(
        "    " + x
        for x in string.split("\n")
    )

def safe_list_get(l, idx):
    if l is None:
        return None
    try:
        return str(l[idx])
    except IndexError:
        return None


In [7]:
def handle_class_builtin(klass) -> str:
    return """
class {class_name}:
    \"\"\"{doc}\"\"\"
{methods}
    """.format(
        class_name=klass.__name__,
        doc=klass.__doc__ or "TODO!: document this",
        methods=tabbify("\n\n".join(
            handle_function_builtin(getattr(klass, x))
            for x in dir(klass)
            if not x.startswith("_")
        )),
    )

import re
def handle_function_builtin(function) -> str:
    args = []
    arg_names = inspect.getfullargspec(function).args
    if "self" in arg_names:
        args.append("self")
    if function.__doc__ is None:
        print('Function %s miss documentation'%function.__name__)
    for line in (function.__doc__ or "").split("\n"):
        if ":" not in line:
            continue
        
        arg_name, _, line = line.partition(":")
        
        if arg_name not in arg_names:
            continue
        
        line = line.strip().strip(",")
        # add auto-defaults for optionals to None if there isn't already one
        if line.startswith("Optional") and "=" not in line:
            line += " = None"
            
        result = "{}: {}".format(
            arg_name,
            line
        )
        args.append(result)
    
    return """
def {function_name}({args}):
    \"\"\"{doc}\"\"\"
""".format(
        function_name=function.__name__,
        doc=tabbify(function.__doc__ or "TODO!: document").strip(),
        args=", ".join(args),
    )
    

In [8]:
def handle_class(klass) -> str:
    return """
class {class_name}({extensions}):
    \"\"\"{doc}\"\"\"
{properties}
{methods}
    """.format(
        class_name=klass.__name__,
        doc=klass.__doc__ or "TODO!: document",
        properties=tabbify("\n\n".join(
            handle_property(x, getattr(klass, x))
            for x in dir(klass)
            if not x.startswith("_") and type(getattr(klass, x)) == property
        )),
        methods=tabbify("\n\n".join(
            handle_function(getattr(klass, x))
            for x in dir(klass)
            if not x.startswith("_") and type(getattr(klass, x)) != property and
            inspect.ismethod(getattr(klass, x))
        )),
        extensions=", ".join(
            x.__name__.replace("embiggen", "grape")
            for x in inspect.getmro(klass)
            if x.__name__ != "object"
        ),
    )

def handle_property(name, function) -> str:
    return """
@property
def {function_name}(self):
    \"\"\"{doc}\"\"\"
""".format(
        function_name=name,
        doc=function.__doc__ or "TODO!: document",
    )
    
    
def handle_function(function) -> str:
    info = inspect.getfullargspec(function)
    args = []
    for (i, arg_name) in enumerate(info.args):
        if arg_name == "self":
            args.append("self")
            continue
            
        result = arg_name
        
        arg_type = getattr(info.annotations.get(arg_name), "__name__", str(info.annotations.get(arg_name)))
        if arg_type is not None:
            if str(arg_type) in BLACKLIST:
                arg_type = '"%s"'%arg_type
            result += ": {}".format(arg_type)
         
        # TODO! figure out how to property do defaults
        # Their order is not trivial (I think that they start form the end)
        #default = safe_list_get(info.defaults, i)
        
        #if default is not None:
        #    result += "= {}".format(default)
            
        args.append(result)
    
    return_annotation = ""
    return_type = info.annotations.get("return")
    if return_type is not None:
        try:
            return_annotation += " -> {}".format(return_type.__name__)
        except AttributeError:
            return_annotation += " -> {}".format(return_type._name)
        
    
    return """
def {function_name}({args}){return_annotation}:
    \"\"\"{doc}\"\"\"
""".format(
        function_name=function.__name__,
        doc=function.__doc__ or "TODO!: document this",
        args=", ".join(args),
        return_annotation=return_annotation,
    )

In [9]:
def imports(f):
    f.write("import typing\n")
    f.write("from typing import *\n")
    f.write("from tensorflow import Tensor, SparseTensor\n") 
    f.write("from ensmallen import Graph\n")     


In [10]:
def build_file_python(path: str, my_object):
    """Write to disk the documentation of the given method or class."""
    directory_path = os.path.dirname(path)
    
    os.makedirs(directory_path, exist_ok=True)
    init_path = os.path.join(directory_path, "__init__.py")
    
    if os.path.exists(init_path):
        with open(init_path, "r") as f:
            init = f.read()
    else:
        init = ""
        
    current_import = "from .{} import *\n".format(os.path.basename(path).replace(".py", ""))
    if current_import not in init:
        init += current_import
        
    for folder in os.listdir(directory_path):
        if "." in folder:
            continue 
            
        import_line = "from . import {}\n".format(folder)
        if import_line not in init:
            init += import_line
        
    with open(init_path, "w") as f:
        f.write(init)

    if inspect.isclass(my_object):
        content = handle_class(my_object)
    else:
        content = handle_function(my_object)
        
    with open(path, "w") as f:
        imports(f)
        f.write(content)

In [11]:
def build_file_builtin(path: str, my_object):
    """Write to disk the documentation of the given method or class."""
    directory_path = os.path.dirname(path)
    
    os.makedirs(directory_path, exist_ok=True)
    init_path = os.path.join(directory_path, "__init__.py")
    
    if os.path.exists(init_path):
        with open(init_path, "r") as f:
            init = f.read()
    else:
        init = ""
        
    current_import = "from .{} import *\n".format(os.path.basename(path).replace(".py", ""))
    if current_import not in init:
        init += current_import
        
    for folder in os.listdir(directory_path):
        if "." in folder:
            continue 
            
        import_line = "from . import {}\n".format(folder)
        if import_line not in init:
            init += import_line
        
    with open(init_path, "w") as f:
        f.write(init)

    if inspect.isclass(my_object):
        content = handle_class_builtin(my_object)
    else:
        content = handle_function_builtin(my_object)
        
    with open(path, "w") as f:
        imports(f)
        f.write(content)

In [12]:
def setup_init_files():
    dirs = [   path
        for path in glob(os.path.join(*ROOT, "**"), recursive=True)
        if os.path.isdir(path)
    ]
    for path in dirs:
        init_path = os.path.join(path, "__init__.py")
        if os.path.exists(init_path):
            continue

        with open(init_path, "w") as f:

            for folder in os.listdir(path):

                if os.path.isdir(os.path.join(path, folder)):
                    f.write("from . import {}\n".format(folder))
                else:
                    print("skipped", folder)
                    continue

    

In [13]:
ROOT = ["skeleton_package", "grape"]

In [14]:
dir(grape)

['EmbeddingResult',
 'Graph',
 'GraphVisualizer',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'datasets',
 'edge_label_prediction',
 'edge_prediction',
 'embedders',
 'embedding_transformers',
 'get_available_models_for_edge_label_prediction',
 'get_available_models_for_edge_prediction',
 'get_available_models_for_node_embedding',
 'get_available_models_for_node_label_prediction',
 'get_models_dataframe',
 'layers',
 'node_label_prediction',
 'sequences',
 'utils',
 'visualizations']

In [15]:
python_stuff = {
    os.path.join(
        *ROOT,
        *inspect.getmodule(v).__name__.split("."), 
        v.__name__
    ) + ".py":v
    for v in recursive_explore_python(grape)
}

In [16]:
rust_stuff = {
    os.path.join(*ROOT, "graph.py"):grape.Graph,
    #os.path.join(*ROOT, "djkstra.py"):ensmallen,
    #os.path.join(*ROOT, "bfs.py"):grape.ShortestPathsResultBFS,
    #**explore_on_level(ensmallen.edge_list_utils, [*ROOT, "ensmallen", "edge_list_utils"]),
    #**explore_on_level(ensmallen.preprocessing, [*ROOT, "ensmallen", "preprocessing"]),
}

In [17]:
python_stuff

{'skeleton_package/grape/datasets/string/CorynebacteriumSpHmsc04h06.py': <function datasets.string.CorynebacteriumSpHmsc04h06(directed=False, preprocess='auto', load_nodes=True, load_node_types=True, load_edge_weights=True, auto_enable_tradeoffs=True, sort_tmp_dir=None, verbose=2, cache=True, cache_path=None, cache_sys_var='GRAPH_CACHE_DIR', version='links.v11.5', **kwargs) -> Graph>,
 'skeleton_package/grape/datasets/string/MicrobacteriumSpCgr1.py': <function datasets.string.MicrobacteriumSpCgr1(directed=False, preprocess='auto', load_nodes=True, load_node_types=True, load_edge_weights=True, auto_enable_tradeoffs=True, sort_tmp_dir=None, verbose=2, cache=True, cache_path=None, cache_sys_var='GRAPH_CACHE_DIR', version='links.v11.5', **kwargs) -> Graph>,
 'skeleton_package/grape/datasets/string/MethanosarcinaSpMtp4Cluster.py': <function datasets.string.MethanosarcinaSpMtp4Cluster(directed=False, preprocess='auto', load_nodes=True, load_node_types=True, load_edge_weights=True, auto_enabl

In [18]:
shutil.rmtree(ROOT[0])

for path, obj in tqdm(python_stuff.items()):
    build_file_python(path, obj)

  0%|          | 0/29800 [00:00<?, ?it/s]

In [19]:

for path, obj in tqdm(rust_stuff.items()):
    build_file_builtin(path, obj)
    
with open(os.path.join(ROOT[0], "setup.py"), "w") as f:
    f.write("""
from setuptools import find_packages, setup

setup(
    name='grape',
    version="1.0.0",
    packages=find_packages(exclude=['contrib', 'docs', 'tests*']),
)
""")
    
setup_init_files()

  0%|          | 0/1 [00:00<?, ?it/s]

skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py
skipped __init__.py


Making sphinx work: 
```
extensions = [
    'sphinx.ext.autodoc',
    'sphinx.ext.todo',
    'sphinx.ext.mathjax',
    'sphinx.ext.githubpages',
]
```


[Reference](https://github.com/davidstutz/sphinx-example/blob/master/)

In [20]:
dir(grape)

['EmbeddingResult',
 'Graph',
 'GraphVisualizer',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'datasets',
 'edge_label_prediction',
 'edge_prediction',
 'embedders',
 'embedding_transformers',
 'get_available_models_for_edge_label_prediction',
 'get_available_models_for_edge_prediction',
 'get_available_models_for_node_embedding',
 'get_available_models_for_node_label_prediction',
 'get_models_dataframe',
 'layers',
 'node_label_prediction',
 'sequences',
 'utils',
 'visualizations']